# Assignment 5: Quantization

In this assignment we are going to look into quantization. Quantisation is one of the non-idealities that we have to consider in digital signal processing. All signal we have been lookin at in previous assignments were already quantized, but you may not have noticed. This is because, in software, we use large data types like float64 that lead to low quantization errors. When capturing signals, in DSP hardware or in resource constrained computing systems, however, we will resort to data types to lower bit resolution.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def uniform_quantize(signal, num_levels):
    min_val, max_val = signal.min(), signal.max()
    q_levels = np.linspace(min_val, max_val, num_levels)
    q_signal = np.digitize(signal, q_levels) - 1
    return q_levels[q_signal]

## Task 1: Software quantization

Until now we always used the numpy standard datatype for numerical arrays, typically float64 for modern systems, to represent data. When we compute on signals, even in software, on systems that are not your typical 64 bit desktop or laptop CPU, we need to use data types of lower precsion (fp32 down to even binary). These can be GPUs, tensor cores, APUs or microcontrollers. Recently, the trend goes towards using low resolution data types even on GPUs (typically fp32) like int8 for data-intensive processes to save memory bandwidth and power.

In this task you need to find a scaling factor (= maximum absolute value) for the signal to maximize its resolution with int8 precision. Remember that int8 means *signed* 8 bit integer and look up its definition.
Then, calculate the quantization error and quantization SNR (SQNR) between *sine* and *quantized_int8*. In this we assume that the fp64 *sine* is the analog signal.

In [ ]:
sampling_rate = 5e6
nyq = sampling_rate / 2
samples = 1024
sine_omega = 2 * np.pi * 5e3
scale = # Insert a scaling factor that maximized the resolution for int8

time = np.arange(samples) / sampling_rate
sine = scale * np.sin(sine_omega * time)
quantized_int8 = sine.astype(np.int8)
bitstring = [f'{x & 0xFF:08b}' for x in quantized_int8]
print("An excerpt of our signal in 8 bit integer format:")
print(bitstring[240:255])

fig_int8, ax_int8 = plt.subplots(figsize=[3.45, 2.3], dpi=200)
ax_int8.plot(1e6*time, sine, drawstyle='steps-mid', label="float64")
ax_int8.plot(1e6*time, quantized_int8, drawstyle='steps-mid', label="int8")
ax_int8.set_xlabel("Time [μs]")
ax_int8.set_ylabel("Amplitude [a.u.]")
ax_int8.set_title("Signal")
ax_int8.legend()

In [ ]:
error = # Calculate the quantization error

sqnr = # Calculate the SQNR in dB based on the formula in lecture 7, slide 6
print(f"The SQNR is {sqnr} dB")

fig_error, ax_error = plt.subplots(nrows=2, sharex=True, figsize=[3.45, 2.3], dpi=200)
ax_error[0].plot(1e6*time, sine)
ax_error[0].plot(1e6*time, quantized_int8, drawstyle='steps-mid')
ax_error[0].set_ylabel("Amplitude [a.u.]")
ax_error[0].set_ylim([118, 128])

ax_error[1].plot(1e6*time, error)
ax_error[1].set_ylabel("Error [a.u.]")
ax_error[1].set_ylim([-0.1, 1.1])

ax_error[0].set_xlim([40, 60])
ax_error[1].set_xlabel("Time [μs]")
ax_error[0].set_title("Quantization error")

### Questions:
- What is the reason for choosing your scale (=max value). What would happen if you go higher? Look at the printed bits for clarification and compare with the int8 defintion.
  - YOUR ANSWER
- Given that you found the scaling for maximum resolution, what is the maximum quantization error in the previous example?
  - YOUR ANSWER

## Task 2: Hardware quantization

From the previous assignment we already now that we want to minimize the sampling rate to save costs on analog-digital-converters (ADCs). The same applies to the amplitude resolution. Typical ADCs have 8 bit to 12 bit resolution, which is already introduces significant quantization errors.

In this task, calculate the error and the quantization SNR.

In [ ]:
bitlevels = [2, 4, 6, 8, 12]

sine = np.sin(sine_omega * time)

fig_mods, ax_quantized = plt.subplots(figsize=[3.45, 2.3], dpi=200)

for bits in bitlevels:
    quantized = uniform_quantize(sine, 2**bits)
    ax_quantized.plot(1e6*time, quantized, drawstyle='steps-mid', label=f"{bits} b")
    error = # Calculate the quantization error
    sqnr = # Calculate the SQNR in dB based on the formula in lecture 7, slide 6
    print(f"The SQNR is {sqnr} dB for {bits} b")

ax_quantized.set_xlabel("Time [μs]")
ax_quantized.set_ylabel("Amplitude [a.u.]")
ax_quantized.set_title("Signal")
ax_quantized.legend()


## Questions:
- What (approximate) relationship can you find between the value of the SQNR and and the number of bits?
  - YOUR ANSWER